In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

<img title="a title" alt="Alt text" src="https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png">

In [3]:
include_points = [23,24,25,26,27,28]
def create_df():
    columns=['Form']

    for pt in include_points:
        landmark = str(mp_pose.PoseLandmark(pt))
        columns.append(landmark[13:]+ '_x')
        columns.append(landmark[13:]+ '_y')

    return pd.DataFrame(columns=columns)

In [4]:
def calulate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle

    return angle

In [5]:
def calculate_disance(a,b):
    a = np.array(a)
    b = np.array(b)

    return np.linalg.norm(a-b)

In [6]:
def record_data(name, label):
    df = create_df()

    cap = cv2.VideoCapture(name)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            if ret==False:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)
            left_shoulder = [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            right_shoulder = [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
            shoulder_distance = calculate_disance(left_shoulder, right_shoulder)
            row=[label]
            for pt in include_points:
                row.append(results.pose_landmarks.landmark[pt].x/shoulder_distance)
                row.append(results.pose_landmarks.landmark[pt].y/shoulder_distance)
            df.loc[len(df)] = row

            #Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            resize = cv2.resize(image, (405, 720))

            # Render detections
            mp_drawing.draw_landmarks(resize, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
            cv2.imshow('MediaPipe Pose', resize)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    return df

In [10]:
correct_df = record_data("vids/correct/2.mp4", "proper")
correct_df.head()

,Form,LEFT_HIP_x,LEFT_HIP_y,RIGHT_HIP_x,RIGHT_HIP_y,LEFT_KNEE_x,LEFT_KNEE_y,RIGHT_KNEE_x,RIGHT_KNEE_y,LEFT_ANKLE_x,LEFT_ANKLE_y,RIGHT_ANKLE_x,RIGHT_ANKLE_y
0,proper,2.868863,2.511873,2.158253,2.488658,2.971478,3.282516,1.941702,3.232102,2.969762,3.951691,1.753330,3.906265
1,proper,2.865515,2.523217,2.164278,2.498843,2.980252,3.299231,1.950216,3.246988,2.983850,3.980998,1.760802,3.933224
2,proper,2.866289,2.541240,2.167863,2.516736,2.998500,3.325141,1.934991,3.269491,3.006666,4.015422,1.760150,3.967448
3,proper,2.888395,2.576733,2.179071,2.553641,3.034345,3.374437,1.932574,3.319376,3.052403,4.076077,1.774323,4.035737
4,proper,2.890922,2.593998,2.171254,2.572569,3.056226,3.399535,1.916078,3.344884,3.081498,4.106230,1.779354,4.071601


In [11]:
incorrect_df = record_data("vids/incorrect/2.mp4", "incomplete")
incorrect_df.head()

,Form,LEFT_HIP_x,LEFT_HIP_y,RIGHT_HIP_x,RIGHT_HIP_y,LEFT_KNEE_x,LEFT_KNEE_y,RIGHT_KNEE_x,RIGHT_KNEE_y,LEFT_ANKLE_x,LEFT_ANKLE_y,RIGHT_ANKLE_x,RIGHT_ANKLE_y
0,incomplete,2.293000,2.099055,1.697144,2.073577,2.436337,2.780282,1.488784,2.737544,2.537123,3.343025,1.378741,3.291633
1,incomplete,2.280865,2.089280,1.686337,2.065568,2.426779,2.761280,1.476668,2.719395,2.518914,3.320630,1.371291,3.270688
2,incomplete,2.288049,2.090803,1.695720,2.067590,2.433908,2.761085,1.477314,2.719456,2.520447,3.323263,1.374242,3.274032
3,incomplete,2.302345,2.087801,1.717425,2.071558,2.434406,2.756071,1.470507,2.714935,2.516461,3.318162,1.373389,3.269027
4,incomplete,2.326325,2.093862,1.738518,2.078318,2.453819,2.768216,1.478604,2.725203,2.530956,3.336640,1.382197,3.287579


In [12]:
correct_df.to_csv("out/csv/correct/2.csv", index=False)
incorrect_df.to_csv("out/csv/incorrect/2.csv", index=False)

<img >